# Notes

code for producing GOIT pipelines summary stats, and for calculating landing page stats

this is saved as an Excel file, which Baird copies/pastes into the existing summary tables information on the drive here:
https://docs.google.com/spreadsheets/d/1OYH6D7c-D0FsL5GzBGijtkmvQCTkBUclj-UVoOieUFo/edit

In [676]:
import pandas
import numpy
import pygsheets
import datetime
import re
import pytz

In [677]:
# define the excel file to save tables in
current_time = datetime.datetime.now(pytz.timezone('US/Eastern')).strftime("%Y-%m-%d_T%H%M%S")

## import data

In [678]:
#fuel_type = 'Gas'
fuel_type = 'Oil'
#fuel_type = 'NGL'

if fuel_type=='Gas':
    excel_writer = pandas.ExcelWriter('GOIT-Summary-Sheets-Gas-'+str(datetime.date.today())+'.xlsx')
    fuel_options = ['Gas']
if fuel_type=='NGL':
    excel_writer = pandas.ExcelWriter('GOIT-Summary-Sheets-NGL-'+str(datetime.date.today())+'.xlsx')
    fuel_options = ['NGL', 
                    'NGL, oil products', 
                    'Oil, NGL', 
                    'Oil, NGL, naphtha']
if fuel_type=='Oil':
    excel_writer = pandas.ExcelWriter('GOIT-Summary-Sheets-Oil-'+str(datetime.date.today())+'.xlsx')
    fuel_options = ['Oil', 
                    'Oil, NGL', 
                    'Oil, NGL, naphtha']

In [679]:
gc = pygsheets.authorize(service_account_env_var='GDRIVE_API_CREDENTIALS')
#spreadsheet = gc.open_by_key('1WaBMIdfRWqSqXUw7_cKXo3RipyhPdnNN8flqEYfMZIA') # file to use for gas pipelines Dec 2023
spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek') # CURRENT sheet

gas_pipes = spreadsheet.worksheet('title', 'Gas pipelines').get_as_df(start='A3')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df(start='A3')

gas_pipes = gas_pipes.drop('WKTFormat', axis=1) # delete WKTFormat column
oil_pipes = oil_pipes.drop('WKTFormat', axis=1)
#pipes_df_orig = gas_pipes.copy() #pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

#get other relevant sheets
country_ratios_df = spreadsheet.worksheet('title', 'Country ratios by pipeline').get_as_df()
owners_df_orig = spreadsheet.worksheet('title', 'Pipeline operators/owners (1/3)').get_as_df(start='A2')
country_ratios_df = country_ratios_df.loc[country_ratios_df.Wiki!='']

# remove empty cells for pipes, owners
if fuel_type=='Gas':
    pipes_df_orig = gas_pipes.copy()
if fuel_type in ['Oil','NGL']:
    pipes_df_orig = oil_pipes.copy()
pipes_df_orig = pipes_df_orig.loc[pipes_df_orig['PipelineName']!='']
#pipes_df_orig = pipes_df_orig.loc[pipes_df_orig['Wiki']!='']
pipes_df_orig = pipes_df_orig.loc[pipes_df_orig.Fuel.isin(fuel_options)]

owners_df_orig = owners_df_orig.loc[owners_df_orig['PipelineName']!='']
#owners_df_orig = owners_df_orig.loc[owners_df_orig['Wiki']!='']
owners_df_orig = owners_df_orig.loc[owners_df_orig.Status!='N/A']
owners_df_orig.set_index('ProjectID', inplace=True)

parent_metadata_df = spreadsheet.worksheet('title', 'Parent metadata (3/3)').get_as_df(start='A2')
parent_metadata_df.set_index('Parent', inplace=True)

In [680]:
country_ratios_df = country_ratios_df.replace('--', numpy.nan)
owners_df_orig = owners_df_orig.replace('',numpy.nan)
owners_df_orig = owners_df_orig.replace('--',numpy.nan)
pipes_df_orig = pipes_df_orig.replace('--',numpy.nan)

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_45894/1118424270.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  country_ratios_df = country_ratios_df.replace('--', numpy.nan)
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_45894/1118424270.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  owners_df_orig = owners_df_orig.replace('--',numpy.nan)
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_45894/1118424270.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be remo

In [681]:
region_df_orig = spreadsheet.worksheet('title', 'Country dictionary').get_as_df(start='A2')

region_name = 'Global'; region_df_touse = region_df_orig.copy()
#region_name = 'AsiaGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.AsiaGasTracker=='Yes']
#region_name = 'EuroGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.EuroGasTracker=='Yes']
#region_name = 'AfricaGasTracker'; region_df_touse = region_df_orig.loc[region_df_orig.AfricaGasTracker=='Yes']
#region_df_agt.copy()

#region_df_touse = region_df_orig.copy()

In [682]:
region_df_touse_cleaned = region_df_touse.loc[(region_df_touse.Region!='--')&
                                            (region_df_touse.SubRegion!='--')]
multiindex_region_subregion = region_df_touse_cleaned.groupby(['Region','SubRegion'])['Country'].count().index
multiindex_region_subregion

MultiIndex([(  'Africa',                 'Northern Africa'),
            (  'Africa',              'Sub-Saharan Africa'),
            ('Americas', 'Latin America and the Caribbean'),
            ('Americas',                'Northern America'),
            (    'Asia',                    'Central Asia'),
            (    'Asia',                    'Eastern Asia'),
            (    'Asia',              'South-eastern Asia'),
            (    'Asia',                   'Southern Asia'),
            (    'Asia',                    'Western Asia'),
            (  'Europe',                  'Eastern Europe'),
            (  'Europe',                 'Northern Europe'),
            (  'Europe',                 'Southern Europe'),
            (  'Europe',                  'Western Europe'),
            ( 'Oceania',       'Australia and New Zealand'),
            ( 'Oceania',                       'Melanesia'),
            ( 'Oceania',                      'Micronesia'),
            ( 'Oceania',

## file names with regional specifics

### create country-specific dataframes for region, country_ratios_df, owners_df

In [683]:
country_ratios_df_touse = country_ratios_df.loc[country_ratios_df['Country'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

owners_df_touse = owners_df_orig.loc[owners_df_orig['Countries'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

pipes_df_touse = pipes_df_orig.loc[pipes_df_orig['Countries'].str.contains(
                                            '|'.join(region_df_touse['Country'].tolist()))]

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_45894/265796674.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  country_ratios_df_touse = country_ratios_df.loc[country_ratios_df['Country'].str.contains(
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_45894/265796674.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  owners_df_touse = owners_df_orig.loc[owners_df_orig['Countries'].str.contains(
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_45894/265796674.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  pipes_df_touse = pipes_df_orig.loc[pipes_df_orig['Countries'].str.contains(


### sum MergedKmByCountry and MergedKmByRegion

In [684]:
status_list = ['proposed', 
               'construction', 
               'shelved', 
               'cancelled', 
               'operating', 
               'idle', 
               'mothballed', 
               'retired']
country_list = sorted(list(set(country_ratios_df_touse['Country'])))
region_list = sorted(list(set(country_ratios_df_touse['Region'])))

In [685]:
excel_status_list = ['proposed', 
                     'construction', 
                     'in development (proposed + construction)', 
                     'shelved', 
                     'cancelled', 
                     'operating', 
                     'idle', 
                     'mothballed', 
                     'retired']
excel_status_list_with_countries = ['Country']+excel_status_list

In [686]:
country_ratios_df_subset = country_ratios_df_touse.loc[country_ratios_df_touse['Fuel'].isin(fuel_options)]

km_by_country = pandas.DataFrame(columns=status_list, index=country_list)
km_by_region = pandas.DataFrame(columns=status_list, index=multiindex_region_subregion)

print('===country-level calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_country[status] = country_ratios_df_subset_status.groupby('Country')['LengthMergedKmByCountry'].sum()

print('===regional calculations===')
for status in status_list:
    print(status)
    country_ratios_df_subset_status = country_ratios_df_subset[country_ratios_df_subset['Status']==status]
    km_by_region[status] = country_ratios_df_subset_status.groupby(['Region','SubRegion'])['LengthMergedKmByCountry'].sum()

# fille NaN with 0.0
km_by_region = km_by_region.fillna(0)
km_by_country = km_by_country.fillna(0)

km_by_region['in development (proposed + construction)'] = km_by_region[['proposed','construction']].sum(axis=1)
km_by_country['in development (proposed + construction)'] = km_by_country[['proposed','construction']].sum(axis=1)

km_by_country = km_by_country[excel_status_list]
km_by_region = km_by_region[excel_status_list]

km_by_region.index.names = ['Region','Subregion']
km_by_country.index.name = 'Country'

km_by_region.loc['Total',:] = km_by_region.sum(axis=0).values
km_by_country.loc['Total',:] = km_by_country.sum(axis=0).values

# drop all-zero rows
km_by_country = km_by_country.loc[~(km_by_country==0).all(axis=1)]

km_by_country.replace(0,'',inplace=True)
km_by_region.replace(0,'',inplace=True)

km_by_region.to_excel(excel_writer, 'Kilometers by region')
km_by_country.to_excel(excel_writer, 'Kilometers by country')

===country-level calculations===
proposed
construction
shelved
cancelled
operating
idle
mothballed
retired
===regional calculations===
proposed
construction
shelved
cancelled
operating
idle
mothballed
retired


/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_45894/2442660946.py:40: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  km_by_region.to_excel(excel_writer, 'Kilometers by region')
/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_45894/2442660946.py:41: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  km_by_country.to_excel(excel_writer, 'Kilometers by country')


In [687]:
km_by_region

proposed construction  \
Region   Subregion                                                
Africa   Northern Africa                      68.5        100.0   
         Sub-Saharan Africa                6482.26       1950.0   
Americas Latin America and the Caribbean     873.0        525.0   
         Northern America                  1255.34       214.38   
Asia     Central Asia                                             
         Eastern Asia                      4467.67       476.28   
         South-eastern Asia                 228.02                
         Southern Asia                       69.47       5265.0   
         Western Asia                      6898.35        175.0   
Europe   Eastern Europe                    1510.34      1237.84   
         Northern Europe                                          
         Southern Europe                    665.07                
         Western Europe                     149.26                
Oceania  Australia and New Zealand                                
         Melanesia                                                
         Micronesia                                               
         Polynesia                                                
Total                                     22667.28       9943.5   

                                         in development (proposed + construction)  \
Region   Subregion                                                                  
Africa   Northern Africa                                                    168.5   
         Sub-Saharan Africa                                               8432.26   
Americas Latin America and the Caribbean                                   1398.0   
         Northern America                                                 1469.72   
Asia     Central Asia                                                               
         Eastern Asia                                                     4943.95   
         South-eastern Asia                                                228.02   
         Southern Asia                                                    5334.47   
         Western Asia                                                     7073.35   
Europe   Eastern Europe                                                   2748.18   
         Northern Europe                                                            
         Southern Europe                                                   665.07   
         Western Europe                                                    149.26   
Oceania  Australia and New Zealand                                                  
         Melanesia                                                                  
         Micronesia                                                                 
         Polynesia                                                                  
Total                                                                    32610.78   

                                          shelved cancelled  operating  \
Region   Subregion                                                       
Africa   Northern Africa                    488.0    620.04    16395.8   
         Sub-Saharan Africa                498.13    2100.0    7457.94   
Americas Latin America and the Caribbean            3937.83   26755.13   
         Northern America                 6481.75  34111.89  111353.42   
Asia     Central Asia                               2048.97    9207.62   
         Eastern Asia                               7144.88   28999.58   
         South-eastern Asia                           306.0    1754.52   
         Southern Asia                              6780.04   25783.24   
         Western Asia                     1544.61   2108.88    21890.4   
Europe   Eastern Europe                              4893.7   55185.79   
         Northern Europe                                       5670.85   
         Southern Europe              

## pipeline km by parent company (owner) and project status

### first check that there are no missing projectids

In [688]:
owner_parent_calculations_df = pandas.DataFrame()
# needs country, km in each country columns as well

for idx,row in country_ratios_df_subset.iterrows():
    #print(row.ProjectID)
    parent_string = row.Parent
    #print(parent_string)

    #print(parent_string)
    # if the first letter of the parent_string is a chinese character, and it ends with [100.00%],
    # that means it's a non-researched QCC owner; keep as-is
    if re.findall(r'[\u4e00-\u9fff]+', parent_string[:1]) != [] and parent_string[-9:]=='[100.00%]':
        parent_list = [parent_string.split(' [100.00%]')[0]]
        percent_list = [1.0]
    # otherwise do the rest of the thing
    else:
        parent_list = re.sub(' \[.*?\]', '', parent_string).split('; ') # all entries must have a Owner [%] syntax
        percent_list = [float(i.rstrip('%'))/100. for i in re.findall('\\d+(?:\\.\\d+)?%', parent_string)]

    if parent_list.__len__()!=percent_list.__len__():
        #print(parent_list)
        if percent_list==[]:
            percent_list = [1/parent_list.__len__() for i in parent_list]
        else:
            nmissing = parent_list.__len__()-percent_list.__len__()
            # distribute nans evenly
            total = numpy.nansum(percent_list)
            leftover = 1-total
            percent_list += [leftover/nmissing]*nmissing
    for p_idx,parent in enumerate(parent_list):
        owner_parent_calculations_df = pandas.concat([owner_parent_calculations_df, 
                                                      pandas.DataFrame([{'Parent':parent, 'ProjectID':row.ProjectID, 
                                                                         'FractionOwnership':percent_list[p_idx],
                                                                         'ParentHQCountry':parent_metadata_df.loc[parent,'ParentHQCountry'],
                                                                         'ParentHQRegion':parent_metadata_df.loc[parent,'ParentHQRegion'],
                                                                         'Country':row.Country,
                                                                         'Status':row.Status,
                                                                         'LengthMergedKmByCountry':row.LengthMergedKmByCountry}])])

owner_parent_calculations_df['KmOwnership'] = owner_parent_calculations_df.FractionOwnership*owner_parent_calculations_df.LengthMergedKmByCountry

In [689]:
owner_parent_calculations_df

,Parent,ProjectID,FractionOwnership,ParentHQCountry,ParentHQRegion,Country,Status,LengthMergedKmByCountry,KmOwnership
0,Enbridge,P0001,1.0000,Canada,Americas,United States,operating,580.70,580.700000
0,Enbridge,P0001,1.0000,Canada,Americas,Canada,operating,1209.30,1209.300000
0,Enbridge,P0002,0.8843,Canada,Americas,Canada,operating,542.35,479.600105
0,23 Indigenous communities,P0002,0.1157,Canada,Americas,Canada,operating,542.35,62.749895
0,Enbridge,P0004,1.0000,Canada,Americas,United States,operating,105.52,105.520000
...,...,...,...,...,...,...,...,...,...
0,SOCAR,P6481,1.0000,Azerbaijan,Asia,Azerbaijan,operating,40.00,40.000000
0,SOCAR,P6482,1.0000,Azerbaijan,Asia,Azerbaijan,operating,43.00,43.000000
0,unknown,P6526,1.0000,unknown,--,United States,construction,32.19,32.190000
0,unknown,P6527,1.0000,unknown,--,United States,construction,32.19,32.190000


In [690]:
#unique_owner_list = owner_parent_calculations_df.Parent.sort_values().unique().tolist()

##################################################
# create km count by owner, status
##################################################
owners_km_by_status_df = \
    owner_parent_calculations_df.groupby(
    ['Parent','ParentHQCountry','Status'])[['KmOwnership']].sum().unstack().droplevel(axis=1, level=[0])

owners_km_by_status_df = owners_km_by_status_df.reindex(columns=status_list)
owners_km_by_status_df = owners_km_by_status_df.reset_index().set_index('Parent')
owners_km_by_status_df.columns.name = None

owners_km_by_status_df['in development (proposed + construction)'] = owners_km_by_status_df[['proposed','construction']].sum(axis=1)

owners_km_by_status_df = owners_km_by_status_df.rename(columns={'Parent':'Parent Company',
                                                                          'ParentHQCountry':'Country'})
# rearrange the order of the columns for output
owners_km_by_status_df = owners_km_by_status_df[excel_status_list_with_countries]

# totals_row = owners_ntrains_by_status_df.sum(axis=0, min_count=0)
# totals_row.name = 'Total'
# owners_ntrains_by_status_df = owners_ntrains_by_status_df.append(totals_row)
owners_km_by_status_df.loc['Total',:] = owners_km_by_status_df.sum(axis=0, min_count=0).values
owners_km_by_status_df.loc['Total','Country'] = ''

owners_km_by_status_df = owners_km_by_status_df.replace(numpy.nan, '')
owners_km_by_status_df = owners_km_by_status_df.replace(0, '')

owners_km_by_status_df.to_excel(excel_writer, sheet_name='Kilometers by owner')

In [691]:
owners_km_by_status_df

,Country,proposed,construction,in development (proposed + construction),shelved,cancelled,operating,idle,mothballed,retired
Parent,,,,,,,,,,
23 Indigenous communities,Canada,,,,,,228.343206,,,
"3Bear Energy, LLC",United States,,,,,96.6,,,,
8 Indigenous Communities,Canada,,,,,,12.825,,,
ADNOC,United Arab Emirates,,,,,,763.11,,,
AMBO Pipeline Limited,United Kingdom,912.15,,912.15,,,,,,
...,...,...,...,...,...,...,...,...,...,...
中国石化集团茂名石油化工有限公司,unknown,,,,,,115.0,,,
中国石油天然气股份有限公司长庆油田分公司,unknown,,,,,,83.7,,,
山东金诚石化集团有限公司,unknown,,,,,,390.0,,,


### pipeline km by start year, type

In [692]:
pipes_started = pipes_df_touse.copy()
#pipes_started['StartYearLatest'].replace(numpy.nan,'',inplace=True)

if fuel_type == 'Gas':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['operating'])) &
                              (pipes_started['Fuel'].isin(fuel_options))]
if fuel_type == 'Oil':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['operating'])) &
                              (pipes_started['Fuel'].isin(fuel_options))]
if fuel_type == 'NGL':
    pipes_started = pipes_started[(pipes_started['Status'].isin(['operating'])) &
                              (pipes_started['Fuel'].isin(fuel_options))]

pipes_started_sum = pipes_started.groupby('StartYearEarliest')['LengthMergedKm'].sum()

In [693]:
pipes_proposed = pipes_df_touse.copy()
#pipes_proposed['StartYearLatest'].replace(numpy.nan,'',inplace=True)

if fuel_type == 'Gas':
    pipes_proposed = pipes_proposed[(pipes_proposed['Status'].isin(['proposed'])) &
                              (pipes_proposed['Fuel'].isin(fuel_options))]
if fuel_type == 'Oil':
    pipes_proposed = pipes_proposed[(pipes_proposed['Status'].isin(['proposed'])) &
                              (pipes_proposed['Fuel'].isin(fuel_options))]
if fuel_type == 'NGL':
    pipes_proposed = pipes_proposed[(pipes_proposed['Status'].isin(['proposed'])) &
                              (pipes_proposed['Fuel'].isin(fuel_options))]

pipes_proposed_sum = pipes_proposed.groupby('ProposalYear')['LengthMergedKm'].sum()

In [694]:
pipes_construction = pipes_df_touse.copy()
#pipes_construction['StartYearLatest'].replace(numpy.nan,'',inplace=True)

if fuel_type == 'Gas':
    pipes_construction = pipes_construction[(pipes_construction['Status'].isin(['construction'])) &
                              (pipes_construction['Fuel'].isin(fuel_options))]
if fuel_type == 'Oil':
    pipes_construction = pipes_construction[(pipes_construction['Status'].isin(['construction'])) &
                              (pipes_construction['Fuel'].isin(fuel_options))]
if fuel_type == 'NGL':
    pipes_construction = pipes_construction[(pipes_construction['Status'].isin(['construction'])) &
                              (pipes_construction['Fuel'].isin(fuel_options))]

pipes_construction_sum = pipes_construction.groupby('ConstructionYear')['LengthMergedKm'].sum()

In [695]:
if fuel_type == 'Gas':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2025)), columns=['Gas pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['Gas pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

if fuel_type == 'Oil':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2025)))
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['Oil pipeline km operating'] = pipes_started_sum
    km_by_start_year['Oil pipeline km construction'] = pipes_construction_sum
    km_by_start_year['Oil pipeline km proposed'] = pipes_proposed_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

if fuel_type == 'NGL':
    km_by_start_year = pandas.DataFrame(index=list(range(1980,2025)), columns=['NGL pipeline km'])
    km_by_start_year.index.name = 'Start year'
    km_by_start_year['NGL pipeline km'] = pipes_started_sum
    km_by_start_year.replace(numpy.nan,0,inplace=True)

km_by_start_year.loc['Total',:] = km_by_start_year.sum(axis=0)

km_by_start_year.to_excel(excel_writer, 'Kilometers by start year')
#km_by_start_year

/var/folders/fl/t07mc8053p33mn6mdmvp45580000gn/T/ipykernel_45894/3403803826.py:23: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  km_by_start_year.to_excel(excel_writer, 'Kilometers by start year')


## save excel file

In [696]:
excel_writer.close()

## calculating stats for landing page

In [697]:
# number of projects tracked in total
print(pipes_df_touse.loc[pipes_df_touse.Fuel.str.contains(fuel_type)].shape[0], fuel_type, 'pipeline projects tracked')
print(pipes_df_touse.loc[pipes_df_touse.Fuel.str.contains(fuel_type)]['LengthMergedKm'].sum()/1e6, 'M km tracked')

1471 Oil pipeline projects tracked
0.4846942 M km tracked


In [698]:
# number of projects tracked in total
print(pipes_df_touse.loc[(pipes_df_touse.Fuel.str.contains(fuel_type))&
                        (pipes_df_touse.Status.isin(['proposed','construction']))].shape[0], fuel_type, 'pipeline projects tracked')
print(pipes_df_touse.loc[(pipes_df_touse.Fuel.str.contains(fuel_type))&
                        (pipes_df_touse.Status.isin(['proposed','construction']))]['LengthMergedKm'].sum()/1e3, 'K km tracked')

167 Oil pipeline projects tracked
38.25467 K km tracked


In [699]:
pipes_df_touse.Fuel.unique()

array(['Oil', 'Oil, NGL', 'Oil, NGL, naphtha'], dtype=object)